In [1]:
!pwd
import sys
sys.executable

/c/Users/shingo/jupyter_notebook/tfgpu_py36_work/02_keras_py/experiment/01_code_test


'C:\\Users\\shingo\\Anaconda3\\envs\\tfgpu20\\python.exe'

In [2]:
%%time
# モジュールimport
import os, sys
sys.path.append(r'C:\Users\shingo\jupyter_notebook\tfgpu_py36_work\02_keras_py')
from tuning import optuna_keras_mlflow as o_k_ml

from tensorflow import keras
import optuna

args = {'output_dir':'output_test/optuna_keras_mlflow'
        , 'n_trials':10
        , 'study_name':'study'
        , 'distribute_name':'model_output'
        , 'is_not_prune':True}

if __name__ == "__main__":
    os.makedirs(args['output_dir'], exist_ok=True)
    study = optuna.create_study(study_name=args["study_name"]
                                , storage=f"sqlite:///{args['output_dir']}/study_history.db"
                                , load_if_exists=True
                                , pruner=optuna.pruners.MedianPruner(n_warmup_steps=10) # MedianPrunerは、あるepochのときで計算したこれまでのtrialの中央値よりも今のepochの値が悪い場合は剪定する。n_warmup_steps=10で最低でも10epochは実行する
                                )

    # 分散実行用にモデルファイルの出力ディレクトリ作成
    args["model_output_dir"] = os.path.join(args['output_dir'], args['distribute_name'])
    os.makedirs(args['model_output_dir'], exist_ok=True)

    objective = o_k_ml.Objective(args)
    study.optimize(objective
                    , n_trials=args["n_trials"]
                    , timeout=600# 600sec=10分経ったらstudy強制終了
                    , callbacks=[objective.mlflow_callback]) #
    study.trials_dataframe().to_csv(f"{args['output_dir']}/study_history.csv", index=False)
    print("Number of finished trials: {}".format(len(study.trials)))
    print(f"\nstudy.best_params:\n{study.best_params}")
    print(f"\nstudy.best_trial:\n{study.best_trial}")

C:\Users\shingo\Anaconda3\envs\tfgpu20\lib\site-packages\nbformat\notebooknode.py:4: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

[I 2020-04-06 00:06:32,502] A new study created with name: study


INFO: 'study' does not exist. Creating a new experiment
INFO: mlflow save dir. [output_test/optuna_keras_mlflow\mlruns\1]


[I 2020-04-06 00:06:38,681] Finished trial#0 resulted in value: 0.5941908319791158. Current best value is 0.5941908319791158 with parameters: {'lr': 0.005413491556280032, 'momentum': 0.11170497111873778}.
[I 2020-04-06 00:06:43,981] Finished trial#1 resulted in value: 1.406791975763109. Current best value is 0.5941908319791158 with parameters: {'lr': 0.005413491556280032, 'momentum': 0.11170497111873778}.
[I 2020-04-06 00:06:49,109] Finished trial#2 resulted in value: 0.5879301998350356. Current best value is 0.5879301998350356 with parameters: {'lr': 0.002207158104210818, 'momentum': 0.7403541967459343}.
[I 2020-04-06 00:06:53,995] Finished trial#3 resulted in value: 0.531676032808092. Current best value is 0.531676032808092 with parameters: {'lr': 0.00559273005319594, 'momentum': 0.7001404456228754}.


Epoch 00060: early stopping


[I 2020-04-06 00:06:56,939] Finished trial#4 resulted in value: 0.016620280138320394. Current best value is 0.016620280138320394 with parameters: {'lr': 0.06978844943760094, 'momentum': 0.37141641909249756}.
[I 2020-04-06 00:07:02,180] Finished trial#5 resulted in value: 1.4241573598649766. Current best value is 0.016620280138320394 with parameters: {'lr': 0.06978844943760094, 'momentum': 0.37141641909249756}.
[I 2020-04-06 00:07:07,444] Finished trial#6 resulted in value: 0.5939711875385708. Current best value is 0.016620280138320394 with parameters: {'lr': 0.06978844943760094, 'momentum': 0.37141641909249756}.


Epoch 00080: early stopping


[I 2020-04-06 00:07:10,957] Finished trial#7 resulted in value: 0.015468465785185495. Current best value is 0.015468465785185495 with parameters: {'lr': 0.0672643890714992, 'momentum': 0.5041147945244098}.
[I 2020-04-06 00:07:16,066] Finished trial#8 resulted in value: 0.02817980729871326. Current best value is 0.015468465785185495 with parameters: {'lr': 0.0672643890714992, 'momentum': 0.5041147945244098}.
[I 2020-04-06 00:07:21,219] Finished trial#9 resulted in value: 0.794050137201945. Current best value is 0.015468465785185495 with parameters: {'lr': 0.0672643890714992, 'momentum': 0.5041147945244098}.


Number of finished trials: 10

study.best_params:
{'lr': 0.0672643890714992, 'momentum': 0.5041147945244098}

study.best_trial:
FrozenTrial(number=7, state=TrialState.COMPLETE, value=0.015468465785185495, datetime_start=datetime.datetime(2020, 4, 6, 0, 7, 7, 491765), datetime_complete=datetime.datetime(2020, 4, 6, 0, 7, 10, 939797), params={'lr': 0.0672643890714992, 'momentum': 0.5041147945244098}, distributions={'lr': LogUniformDistribution(high=0.1, low=1e-05), 'momentum': UniformDistribution(high=1.0, low=0.0)}, user_attrs={'batch_size': 16, 'data': 'sklearn.datasets.load_wine', 'normalize': 'StandardScaler', 'optimizer': 'SGD', 'test_size': 0.25}, system_attrs={'_number': 7}, intermediate_values={}, trial_id=8)
